In [26]:
import numpy as np
#import math
#import scipy
#import scipy.linalg  
#import time
#import scipy.sparse as sparse
#import scipy.sparse.linalg
import matplotlib
import matplotlib.pyplot as plt
#import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
#from pyDOE import lhs
import tensorflow as tf


#import tensorflow_probability as tfp

#Log: Implementing Eliptical PINN ADAM Aug 2023

Burgers Equation

uy+uux= = 0, y positive

solution: u(x, y) = x/(1+y)

Eliptical Equation with Boundary Condition

u_yy+u_xx=0, x in [0,1], y in [0,1]



u(0,x) = sin(c  x)

u(1, x) = sin(c x)*exp(c)

u(y, 0) =0

u(y,1) = sin(c)exp(cy)

Exact Solution

u(x, y) = sin(c x)exp(c y)

uy = c sin(c x)exp(cy)

uyy = c^2 sin(c x) exp(c y)


ux = c cos(c x) exp(c y)

uxx = -c^2 sin(c x) exp(c y)


uxx+uyy = 0

In [27]:
#exact solution
def u_exact(data):
    output = tf.math.sin(const*data[:, 0])*tf.math.sin(const*data[:, 1])
    return output

In [28]:
#exact laplacian
def f_exact(data):
    output = -(2*const**2)*u_exact(data)
    return output

In [29]:
def p2t(x,y):
    #point to tensor for inputs
    xin = np.atleast_1d(x)
    yin = np.atleast_1d(y)
    output = tf.constant(np.array([xin,yin]).T, dtype=tf.float32)
    return output

In [30]:
def gen_whole_inputs(xmin, xmax, ymin, ymax, xnum, ynum):
    x = np.linspace(xmin, xmax, xnum)
    y = np.linspace(ymin, ymax, ynum)
    xv, yv = np.meshgrid(x, y)
    xv = np.concatenate(xv)
    yv = np.concatenate(yv)
    data = p2t(xv, yv)
    return data

In [31]:
def gen_bound_input(xmin, xmax, ymin, ymax, xnum, ynum):
    xline = np.linspace(xmin, xmax, xnum)
    yline = np.linspace(ymin, ymax, ynum)
    top = p2t(xline, np.ones(xnum)*ymax)
    bottom = p2t(xline, np.ones(xnum)*ymin)
    left = p2t(np.ones(ynum)*xmin, yline)
    right = p2t(np.ones(ynum)*xmax, yline)
    output = tf.concat([top, bottom, left, right], 0)
    output = tf.raw_ops.UniqueV2(x=output, axis = [0])[0]
    return output

In [32]:
def get_model():
        model=Sequential([Dense(width,activation='tanh',
                                kernel_initializer=tf.keras.initializers.GlorotNormal(),
                                input_shape=(2,),name='H1')])
        for i in range(depth):
            model.add(Dense(width,activation='tanh',name='H'+str(i+2)))
        model.add(Dense(1,name='output_layer'))
        return model

In [33]:
def u(model, data):
    output = model(data)
    return output[:, 0]

In [34]:
def f(model, data):
    with tf.GradientTape(persistent = True) as tp1:
        tp1.watch(data)
        with tf.GradientTape() as tp2:
            tp2.watch(data)
            sol = u(model, data)
        grad= tp2.gradient(sol, data)
        u_x = grad[:, 0]
        u_y = grad[:, 1]
    u_xx = tp1.gradient(u_x, data)[:, 0]
    u_yy = tp1.gradient(u_y, data)[:,1]
    output = u_xx+u_yy
    return output 

In [35]:
def ode_system(model, u_hat, bound_init, bddata, intdata, f_exact_int, u_hat_bound): 
    PINN_loss = tf.reduce_mean(tf.math.square(f_exact_int-f(model, intdata)))
    MSE_loss = tf.reduce_mean(tf.math.square(u_hat_bound-u(model, bddata)))
    total_loss = PINN_loss+MSE_loss
    
    return total_loss

In [36]:
def train_model(model, u_hat, bound_init, bddata, intdata, max_it):
    loss_history = []
    f_exact_int = f_exact(intdata)
    u_hat_bound = u_hat(bound_init, bddata)
    for itr in range(max_it):
        with tf.GradientTape() as tape:
            train_loss = ode_system(model,u_hat, bound_init, bddata, intdata, f_exact_int, u_hat_bound)
            grads = tape.gradient(train_loss, model.trainable_variables)
            optm.apply_gradients(zip(grads, model.trainable_variables))

        if itr % 10 == 0:
            print(train_loss.numpy())
            loss_history.append(train_loss.numpy())
    return model, loss_history

In [37]:
def train_course(model,fine_models, lambda_f, max_it):
    c_loss_history = []
    
    bddata = gen_bound_input(0, 1, 0, 1, sub_Nvm, sub_Nvn)
    intdata = gen_whole_inputs(0, 1, 0, 1, sub_Nvm, sub_Nvn)
    exact_bounds = u_exact(bddata)
    u_fine_int = predictor(fine_models, intdata)
    f_exact_int = f_exact(intdata)
    for itr in range(max_it):
        with tf.GradientTape() as tape:
            train_loss = c_ode_system(model, fine_models, bddata, intdata, lambda_f, exact_bounds, u_fine_int, f_exact_int)
            grads = tape.gradient(train_loss, model.trainable_variables)
            optm.apply_gradients(zip(grads, model.trainable_variables))
        if itr % 10 == 0:
            print(train_loss.numpy())
            c_loss_history.append(train_loss.numpy())
    return model, c_loss_history

In [38]:
def c_ode_system(model, fine_models, bddata, intdata, lambda_f, exact_bounds, u_fine_int, f_exact_int):
    int_loss = tf.reduce_mean(tf.math.square(f_exact_int-f(model, intdata)))
    
    b_loss = tf.reduce_mean(tf.math.square(exact_bounds-u(model, bddata)))
    
    fine_loss = tf.reduce_mean(tf.math.square(u(model, intdata)-u_fine_int))
    
    total_loss = int_loss+b_loss+lambda_f*fine_loss
    return total_loss


In [39]:

def subdomain_locator(Nvm, Nvn, x, y):
    intx = x*Nvm
    inty = y*Nvn

    iintx=int(intx)
    iinty=int(inty)
    
    colnum= np.sign(intx-iintx)*1+iintx+1-abs(np.sign(intx))

    rownum= np.sign(inty-iinty)*1+iinty+1-abs(np.sign(inty))

    i = int(colnum-1+(rownum-1)*Nvm)
    #if i!=0:
    #    print(i, x, y, colnum, rownum)
    return i

In [40]:
def predictor(models, indata):
    output = np.zeros(len(indata[:,0]))
    for i in range(len(indata)):
        datax = indata[i, 0]
        datay = indata[i, 1]
        el = subdomain_locator(Nvm, Nvn, datax, datay)
        model = models[el]
        
        #input_data = np.vstack([datapoint, [0,0]])

        #input_data = tf.convert_to_tensor(input_data, np.float32)
        #tensorpoint = tf.Variable([datapoint], dtype=tf.float32)
        out = model(p2t(datax, datay))
        outarray = out.numpy()
        output[i] = outarray
        
        
        #output[i] = u_exact(datapoint) #for testing
    
        
        
   
    return output

In [41]:
def update_bounds_coarse(u_coarse, lambda_c, models, DomNum, sub_x_min, sub_x_max, sub_y_min, sub_y_max, sub_Nvm, sub_Nvn, top_bounds, bottom_bounds, left_bounds, right_bounds, bound_init):
    
    for i in range(DomNum):
        xline = np.linspace(sub_x_min[i], sub_x_max[i], sub_Nvm)
        yline = np.linspace(sub_y_min[i], sub_y_max[i], sub_Nvn)
        if sub_y_min[i] !=0:
            indata = p2t(xline, np.full(len(xline), sub_y_min[i]))
            bottom_bounds[i, :] = (1-lambda_c)*predictor(models, indata)+lambda_c*u_coarse(indata).numpy()[:,0]
        if sub_y_max[i] !=1:
            indata = p2t(xline, np.full(len(xline), sub_y_max[i]))
            top_bounds[i, :] = (1-lambda_c)*predictor(models, indata)+lambda_c*u_coarse(indata).numpy()[:,0]
        if sub_x_min[i] !=0:
            indata = p2t(np.full(len(yline), sub_x_min[i]), yline)
            left_bounds[i, :] = (1-lambda_c)*predictor(models, indata)+lambda_c*u_coarse(indata).numpy()[:,0]
        if sub_x_max[i] !=1:
            indata = p2t(np.full(len(yline), sub_x_max[i]), yline)
            right_bounds[i,:] = (1-lambda_c)*predictor(models, indata)+lambda_c*u_coarse(indata).numpy()[:,0]

    return (top_bounds, bottom_bounds, left_bounds, right_bounds)

In [42]:
def Eliptic_PINN(model, u_hat, bound_init, top_bounds, bottom_bounds, left_bounds, right_bounds, xmin, xmax, ymin, ymax, sub_Nvm, sub_Nvn):
    
    
    bound_data = gen_bound_input(xmin, xmax, ymin, ymax, sub_Nvm, sub_Nvm)
    whole_data = gen_whole_inputs(xmin, xmax, ymin, ymax, sub_Nvm, sub_Nvn)
    
    [model, sub_loss] = train_model(model,u_hat, bound_init, bound_data, whole_data, max_it)

    
    return model, sub_loss

In [43]:
def DeepDDM(Nvm,Nvn, h_x, h_y, epochs, sub_Nvm, sub_Nvn, bound_init):
    #Nvm = number of sub domains on x  axis
    #Nvn = number of sub domains  on y axis
        #1 on both means there is only the original domain
    
    #sub_Nvm = number of elements within subdomains on  x axis for use in the net training 
    #sub_Nvn = number of elements within subdomains on  y axis for use in the net training
        #1 on both means just corners would be used
    
    
    #Nvm*Nvn= number of subdomains
    #h is percentage of overlap in the relevant axis
        #note h_x<=1, h_y<=1
    #Space is [0,1] by [0,1]
    
    #connnectivity matrix, as in what subdomains are connected
    #is boundary matrix
    #range of the various subdomains
    
    len_x = 1/(Nvm) # length of the subdivisions on x axis
    len_y = 1/(Nvn) #length of the subdivisions on y axis
    global domain_loss
    global l2max
    l2max = []
    DomNum = Nvm*Nvn
    sub_x_min = np.empty(DomNum) 
    sub_x_max = np.empty(DomNum)
    sub_y_min = np.empty(DomNum)
    sub_y_max = np.empty(DomNum)
    el_rownum = np.empty(DomNum)
    el_colnum = np.empty(DomNum)
    
    if 0==Nvm-1:
        sub_x_max = np.ones(DomNum)
        sub_x_min = np.zeros(DomNum)
    else:
        for i in range(DomNum):
        #generates which row, column each element is in
            el_colnum[i] = np.mod(i,Nvm)
            if el_colnum[i] == 0:
                sub_x_max[i] = (el_colnum[i]+1+h_x)*len_x
                sub_x_min[i] = (el_colnum[i])*len_x
            elif el_colnum[i] == Nvm-1:
                sub_x_min[i] = (el_colnum[i]-h_x)*len_x
                sub_x_max[i] = (el_colnum[i]+1)*len_x
            else:
                sub_x_min[i] = (el_colnum[i]-h_x)*len_x
                sub_x_max[i] = (el_colnum[i]+1+h_x)*len_x

    if 0==Nvn-1:
        sub_y_max = np.ones(DomNum)
        sub_y_min = np.zeros(DomNum)
    else:
        for i in range(DomNum):
        #generates which row, column each element is in
            el_rownum[i] = int(i/Nvm) 

            if el_rownum[i]==0:
                sub_y_max[i] = (el_rownum[i]+1+h_y)*len_y
                sub_y_min[i] = el_rownum[i]*len_y
            elif el_rownum[i]==Nvn-1:
                sub_y_min[i] = (el_rownum[i]-h_y)*len_y
                sub_y_max[i] = (el_rownum[i]+1)*len_y
            else:
                sub_y_min[i] = (el_rownum[i]-h_y)*len_y
                sub_y_max[i] = (el_rownum[i]+1+h_y)*len_y

        
    u_pred = np.zeros([Nvm*sub_Nvm, Nvn*sub_Nvn])    
    top_bounds = np.zeros([DomNum , sub_Nvm])
    bottom_bounds = np.zeros([DomNum , sub_Nvm]) 
    left_bounds = np.zeros([DomNum, sub_Nvn]) 
    right_bounds = np.zeros([DomNum, sub_Nvn]) 
    
    models = [None]*DomNum
    dom_loss_history = np.zeros(epochs)
    coarse_loss_history = np.zeros(epochs)
    
    #initialize boundaries
    xzeros = np.zeros(sub_Nvm)
    yzeros = np.zeros(sub_Nvn)
    xones = np.ones(sub_Nvm)
    yones = np.ones(sub_Nvn)
    u_coarse = get_model()
    for i in range(DomNum):
        xline = np.linspace(sub_x_min[i], sub_x_max[i], sub_Nvm)
        yline = np.linspace(sub_y_min[i], sub_y_max[i], sub_Nvn)
        
        if sub_y_min[i] ==0:
            bottom_bounds[i, :] = u_exact(p2t(xline, xzeros))
            #for j in range(sub_Nvm):
             #   bottom_bounds[i, j] = u_exact(p2t(xline[j], 0))
        else:
            bottom_bounds[i, :] = bound_init(p2t(xline, xzeros))
            #for j in range(sub_Nvm):
            #   bottom_bounds[i, j] = bound_init([xline[j], sub_y_min[i]])
        if sub_y_max[i] ==1:
            top_bounds[i, :] = u_exact(p2t(xline, xones))
            #for j in range(sub_Nvm):
            #    top_bounds[i,j] = u_exact(p2t(xline[j], 1))
        else:
            top_bounds[i, :] = bound_init(p2t(xline, xones))

            #for j in range(sub_Nvm):
                #top_bounds[i,j] = bound_init([xline[j], sub_y_max[i]])

        if sub_x_min[i]==0:
            left_bounds[i, :] = u_exact(p2t(yzeros, yline))
            #for j in range(sub_Nvn):
            #    left_bounds[i, j] =  u_exact(p2t(0, yline[j]))
        else:
            left_bounds[i, :] = bound_init(p2t(yzeros, yline))
            #for j in range(sub_Nvn):
            #   left_bounds[i, j] = bound_init([sub_x_min[i], yline[j]])
                
        if sub_x_max[i]==1:
            right_bounds[i, :] = u_exact(p2t(yones, yline))
            #for j in range(sub_Nvn):
            #    right_bounds[i, j] = u_exact(p2t(1, yline[j]))
        else:
            right_bounds[i, :] = bound_init(p2t(yones, yline))
            #for j in range(sub_Nvn):
            #    right_bounds[i, j] = bound_init([sub_x_max[i], yline[j]])
   
    #exact_l2 = f_exact(gen_whole_inputs(0, 1, 0, 1, 100, 100))
    error_history = np.zeros(epochs)
    #xl2 = np.linspace(0, 1, 100)
    #yl2 = np.linspace(0, 1, 100)

    #Xl2, Yl2 = np.meshgrid(xl2,yl2) 
    #Xl2_star=np.hstack((Xl2.flatten()[:,None],Yl2.flatten()[:,None]))
    
    
    def u_hat(bound_init, data):
        approx = bound_init(data)
        if type(approx) == np.ndarray:
            output = approx
        else:
            output = approx.numpy()
        for i in range(len(data)):
            if data[i, 0]==0 or data[i, 1]==0 or data[i, 1]==1 or data[i, 1]==1:
                output[i] = u_exact(tf.expand_dims(data[i, :], axis = 0))

        return output
    for i in range(DomNum):
        models[i] = get_model()
        
    tol_data = gen_bound_input(0, 1, 0, 1, 100, 100)
    flag=False
    epoch=0
    while epoch<epochs and flag==False: 
        if epoch ==1: #the second epoch, updates the initializer to be the predictor, so boundary info exchange happens
            def bound_init2(data):
                output= predictor(models, data)
                return output
            bound_init = bound_init2

        for i in range(DomNum):
            print("Fine Epoch: "+str(epoch+1) +"  Subdomain: "+str(i+1))
            (update_model, loss_update) = Eliptic_PINN(models[i], u_hat,bound_init,  top_bounds[i, :], bottom_bounds[i, :], left_bounds[i, :], right_bounds[i, :], sub_x_min[i], sub_x_max[i], sub_y_min[i], sub_y_max[i], sub_Nvm, sub_Nvn)
            models[i] = update_model 
            
            dom_loss_history[epoch] = dom_loss_history[epoch]+loss_update[-1]
        #Xl2_star=np.hstack((Xl2.flatten()[:,None],Yl2.flatten()[:,None]))
        lambda_f=0.5
        print("Coarse Epoch: ", epoch+1)
        (u_coarse, c_loss_update) = train_course(u_coarse, models, lambda_f, max_it)
        coarse_loss_history[epoch] += c_loss_update[-1]
        error_history[epoch] = get_l2(models)
        
        
        #network tolerance criteria for stopping
        if epoch==0:
            prev_net = predictor(models, tol_data)
            prev_norm = tf.reduce_mean(tf.math.square(prev_net))
        else:
            new_net = predictor(models, tol_data)
            net_diff = tf.reduce_mean(tf.math.square(prev_net -new_net))
            new_norm = tf.reduce_mean(tf.math.square(new_net))
            rel_net_diff = net_diff/min(new_norm, prev_norm)
            if rel_net_diff<net_tol:
                print("Network has converged")
                flag=True
            else:
                prev_net = new_net
                prev_norm = new_norm
                
        if error_history[epoch]<error_tol:
            flag = True
            print("Error is below tolerance.")
                
                
        
        
        
        #update bounds with new guesses   
        lambda_c = 0.9*(coarse_decay)**epoch
        (new_top_bounds, new_bottom_bounds, new_left_bounds, new_right_bounds)= update_bounds_coarse(u_coarse, lambda_c, models, DomNum, sub_x_min, sub_x_max, sub_y_min, sub_y_max, sub_Nvm, sub_Nvn, top_bounds, bottom_bounds, left_bounds, right_bounds, bound_init)
        
        
        
        
        epoch+=1
    
    dom_loss_history = dom_loss_history/(DomNum) #Averages over subdomains
    
    
    plt.figure()
    for i in range(DomNum):
        plt.vlines(x=[sub_x_min[i], sub_x_max[i]], ymin=0, ymax=1)
        plt.hlines(y=[sub_y_min[i], sub_y_max[i]], xmin=0, xmax=1)
    plt.show()
    
    plt.figure()
    plt.plot(dom_loss_history[np.nonzero(dom_loss_history)], label="Fine Loss")
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(coarse_loss_history[np.nonzero(coarse_loss_history)], label="Coarse Loss")
    plt.legend()
    plt.show()
    
    plt.figure()
    plt.plot(error_history[np.nonzero(error_history)], label ="Relative L2 Error")
    plt.legend()
    plt.show()
    
    
    
    print("Fine Loss: "+str(dom_loss_history[np.nonzero(dom_loss_history)]))
    print("Coarse Loss: "+ str(coarse_loss_history[np.nonzero(coarse_loss_history)]))
    print("Error " +str(error_history[np.nonzero(error_history)]))
    np.savetxt("C_Loss_and_error.txt", (dom_loss_history[np.nonzero(dom_loss_history)],coarse_loss_history[np.nonzero(coarse_loss_history)], error_history[np.nonzero(error_history)]), delimiter =", ", header= "Fine Loss, Coarse Loss, Error")
    
    return models
    
    
        
        
            
            
    

In [44]:
def get_l2(models):
    data = gen_whole_inputs(0,1, 0,1, 100, 100)
    pred = predictor(models, data)
    exact = u_exact(data)
    #pred = pred.reshape([100, 100])
    error = np.linalg.norm(exact-pred)/np.linalg.norm(exact)
    return error


In [45]:
def plotter(models):
    data = gen_whole_inputs(0, 1, 0, 1, 100, 100)
    approx = predictor(models, data)
    exact = u_exact(data).numpy()
    abs_error = abs(exact - approx)
    MAE_error = (abs_error).mean()
    print("Mean Absolute Error: " + str(MAE_error))
    rel_L2_err = np.linalg.norm(exact-approx)/np.linalg.norm(exact) 
    print("Relative L2 Error: " + str(rel_L2_err))
    print("Error Max "+ str(abs_error.max()))
    print("Error Min "+ str(abs_error.min()))
    x = data[:, 0].numpy()
    y = data[:, 1].numpy()
    
    plt.figure(figsize=(12,12))
    ax = plt.subplot(3, 1, 1)
    plt.tricontourf(x, y, abs_error ,100)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.colorbar(label = "Absolute Error")
    #plt.plot()
    
    plt.subplot(3, 1, 2)
    plt.tricontourf(x, y, approx ,100)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.colorbar(label = "Predicted")
    np.savetxt("C_Approx.txt", approx)
    
    plt.subplot(3, 1, 3)
    plt.tricontourf(x, y, exact ,100)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.colorbar(label = "Exact")
    np.savetxt("C_Exact.txt", exact)

    
    return 

In [46]:
def xsplice_plt(models, x):
    y = np.linspace(0, 1, 100) 
    data= p2t(np.full(100, x), y)
    exact = u_exact(data)
    
    plt.plot(exact, label = "X = "+str(x))
    plt.plot(predictor(models, data), label = "X Approx")
    plt.legend()
    plt.show()


In [47]:
def ysplice_plt(models, y):
    x = np.linspace(0, 1, 100) 

    data = p2t( x, np.full(100, y))


    plt.plot(u_exact(data), label = "Y = "+str(y))
    plt.plot(predictor(models, data), label = "Y Approx")
    plt.legend()
    plt.show()

In [48]:
def zero_init(data):
    return np.zeros(len(data))
#for exact initializer put u_exact as the initializer
#def random_init(data):
#    return np.random.rand(len(data))
def avg_init(x, y):
    avg = 0.25*((1-x)*u_exact([x, 0])+x*u_exact([x, 1])+(1-y)*u_exact([0, y])+y*u_exact([1, y]))
    return avg
#def noisy_init(data):
 #   return u_exact(data)+noise*random_init(data)

In [49]:
Nvm = 3 #number of subintervals along x axis
Nvn = 3 #number of subintervals along y axis
h_x = 0.2 #percentage of overlap (<=1)
h_y = 0.2 #percentage of overlap (<=1)
epochs = 100
sub_Nvm = 100 #mesh for PINN training
sub_Nvn = 100
max_it = 100 #max_it for PINN training
initializer = zero_init #u_exact, zero_init, random_init, avg_init, noisy_init
global noise
noise = 0.05 #only for noisy initializer
#reset_models_betweeen_epochs = True
optm = tf.keras.optimizers.legacy.Adam(amsgrad=True)
net_tol= 1e-3 #tolerance for network convergeance 
error_tol = 0.1 #tolerance for error

coarse_decay = 0.8 #between 0 and 1. Adjust how fast coarse network influence decays


global per_collocation
global per_interior
global width #width in NN models
global depth #depth in NN models
global const #PDE constant
global int_weight #weight for interior loss, between 0 and 1, higher means more weight on PDE loss than boundary loss
int_weight = 0.5
const = 3.141592653589793 #math.pi
per_collocation = 0.5
per_interior = 0.5
width  = 20
depth  = 6

In [50]:
tf.random.set_seed(1234)
models = DeepDDM(Nvm, Nvn, h_x, h_y, epochs, sub_Nvm, sub_Nvn, initializer)

Fine Epoch: 1  Subdomain: 1
69.9157
40.260094
19.255463
15.728298
11.260887
6.774018
4.0517683
2.0685124
1.1366857
1.0402411
Fine Epoch: 1  Subdomain: 2
119.65555
35.71056
34.235443
28.362703
20.976774
4.539829
3.8104527
2.7335536
1.8564831
1.2979535
Fine Epoch: 1  Subdomain: 3
58.215675
20.997702
12.849425
10.066545
5.9596395
3.3939857
2.4034824
2.0924366
1.8383036
1.6708257
Fine Epoch: 1  Subdomain: 4
122.89459
29.729536
4.7555437
3.7436066
1.7778428
1.4442706
1.1304113
1.0564218
0.9677651
0.9138119
Fine Epoch: 1  Subdomain: 5
267.0264
76.6399
25.90387
8.670925
5.5721936
3.2254362
2.7760353
2.3326979
2.0687602
1.866673
Fine Epoch: 1  Subdomain: 6
113.69914
20.462875
7.9107003
5.5676827
3.900395
3.3014522
2.964855
2.7858005
2.6324449
2.4627929
Fine Epoch: 1  Subdomain: 7
58.479477
12.7980175
8.776554
4.078543
2.9886875
1.6518528
1.4232571
1.2170995
1.0955406
0.9913742
Fine Epoch: 1  Subdomain: 8
131.50656
37.081676
11.365423
3.5653284
2.8738053
2.3261456
1.9275215
1.6733215
1.4657041


0.20497537
0.2005482
0.19609566
0.19161762
0.18711466
Fine Epoch: 7  Subdomain: 3
0.012884539
0.012515135
0.01220585
0.011981096
0.011809421
0.011668699
0.011550185
0.011449687
0.011363491
0.011288596
Fine Epoch: 7  Subdomain: 4
0.14115122
0.13919538
0.1372061
0.1354597
0.13384484
0.132271
0.13071583
0.1291789
0.12765919
0.12615398
Fine Epoch: 7  Subdomain: 5
0.4809589
0.4752258
0.46946925
0.46377814
0.4581595
0.4526038
0.44710356
0.44165605
0.43626043
0.43091607
Fine Epoch: 7  Subdomain: 6
0.031195562
0.030829158
0.030489473
0.03019362
0.02993128
0.029689249
0.029462213
0.029249769
0.029050061
0.028861724
Fine Epoch: 7  Subdomain: 7
0.043720555
0.0409035
0.038406637
0.03621038
0.03426654
0.032550216
0.031033386
0.029688008
0.028489243
0.027414665
Fine Epoch: 7  Subdomain: 8
0.083541416
0.081766136
0.08005705
0.07840629
0.076815456
0.07527883
0.07379275
0.07235336
0.0709576
0.06960292
Fine Epoch: 7  Subdomain: 9
0.06151115
0.061483324
0.14769909
0.14155255
0.06531217
0.057057153
0.0568

KeyboardInterrupt: 

In [ ]:
plotter(models)

In [ ]:
xsplice_plt(models, 0.5)

In [ ]:
ysplice_plt(models, 0.50)